<a href="https://colab.research.google.com/github/1985shree/understanding-embeddings-with-word2vec/blob/main/Understand_embeddings_with_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>
<hr>

_Embeddings_ are representation of words thanks to vectors. These embeddings can be learnt within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which the ÌMDB dataset: it corresponds to sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [ ]:
!pip install --quiet nbresult
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/data-challenges-master/06-Deep-Learning/05-Natural-Language-Processing/02-Embeddings-with-Word2vec/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

In the previous exercise, we jointly learnt a representation for the words, and feed this representation to a RNN, as shown here : 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which can slow the convergence, and make it harder!

For that reason, we will separate the steps of learning the word representation and feeding it to a RNN. As shown here : 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with the word2vec.

The drawback is indeed that the learnt embedding is not _specifically_ designed for our task. However, learning it independently of the task at hand (sentiment analysis) has some advantages : 
- it is very fast to do in general (with word2vec)
- the representation learnt by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As on this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`.

In [ ]:
pip install gensim python-Levenshtein

In [ ]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)

Let's look at the embedded representation of some words.

You can use `word2vec.wv` as a dictionary.
For instance, `word2vec.wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation were not learn). 

In [ ]:
word2vec.wv.most_similar('glass', topn=10)

[('python', 0.990651547908783),
 ('tim', 0.989586353302002),
 ('pre', 0.989538848400116),
 ('texas', 0.9889823794364929),
 ('blake', 0.9888622760772705),
 ('vincent', 0.9886792898178101),
 ('stereotypical', 0.9884096384048462),
 ('elegant', 0.9883958697319031),
 ('blade', 0.9882690906524658),
 ('combining', 0.9879271984100342)]

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [ ]:
word2vec.vector_size

100

How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the `word2vec.most_similar(...)` method that, given an input word, display the "closest" words in the embedding space. If the embedding is well done, then words that have close meanings will have close representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [ ]:
word2vec.wv.most_similar('glass', topn=10)

[('python', 0.990651547908783),
 ('tim', 0.989586353302002),
 ('pre', 0.989538848400116),
 ('texas', 0.9889823794364929),
 ('blake', 0.9888622760772705),
 ('vincent', 0.9886792898178101),
 ('stereotypical', 0.9884096384048462),
 ('elegant', 0.9883958697319031),
 ('blade', 0.9882690906524658),
 ('combining', 0.9879271984100342)]

Similarly to `most_similar` used on words directly, we can use `similar_by_vector` on vectors to do the same thing :

In [ ]:
word2vec.wv.similar_by_vector('glass', topn=10)

[('python', 0.990651547908783),
 ('tim', 0.989586353302002),
 ('pre', 0.989538848400116),
 ('texas', 0.9889823794364929),
 ('blake', 0.9888622760772705),
 ('vincent', 0.9886792898178101),
 ('stereotypical', 0.9884096384048462),
 ('elegant', 0.9883958697319031),
 ('blade', 0.9882690906524658),
 ('combining', 0.9879271984100342)]

# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [ ]:
word2vec.wv['good'] -  word2vec.wv['bad']

array([ 0.04518814,  0.21798527, -0.10849094, -0.15209422, -0.18512094,
       -0.27149183,  0.00163224,  0.4410518 , -0.23591161,  0.01391269,
       -0.00809795,  0.05873898, -0.39261344, -0.33025366, -0.4270484 ,
       -0.26625407, -0.31040418,  0.27404058, -0.48514533,  0.25722885,
       -0.4649549 ,  0.43236205, -0.332506  , -0.01274824,  0.053725  ,
        0.1098669 , -0.00632368, -0.36800712, -0.05635104,  0.14885473,
       -0.16096309, -0.55898976, -0.06341916,  0.0190593 , -0.41015193,
       -0.02794771,  0.12055682, -0.37423015,  0.29607034,  0.41876784,
       -0.7030258 ,  0.2343294 , -0.31386822,  0.5940865 ,  0.10477261,
        0.15247157, -0.23764764, -0.31835485, -0.21603478, -0.14210874,
        0.15350693, -0.11302818, -0.0987795 , -0.04970437,  0.20921552,
       -0.5819511 , -0.00840309, -0.36096555,  0.2378102 , -0.34124297,
       -0.47562972,  0.21016866, -0.38906604,  0.05913702,  0.11685246,
        0.17954521, -0.38297546,  0.14285791,  0.5176842 ,  0.39

Now, image for a second that, somehow, the following equality holds true - just for a second

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to 

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be foolish of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [ ]:
res = word2vec.wv['good'] -  word2vec.wv['bad'] + word2vec['stupid']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


We earlier said that, for any vector, it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `word2vec.wv.similar_by_vector` function) of `res`

In [ ]:
word2vec.wv.similar_by_vector(res)

[('nice', 0.7795281410217285),
 ('given', 0.7698271870613098),
 ('destined', 0.7664583921432495),
 ('decent', 0.7643337249755859),
 ('gunslinger', 0.7638342976570129),
 ('such', 0.7596216201782227),
 ('goofy', 0.7548052072525024),
 ('potential', 0.753576397895813),
 ('smart', 0.7521515488624573),
 ('generous', 0.7513566613197327)]

In [ ]:
word2vec.wv['boy'] -  word2vec.wv['girl'] 

array([ 1.45892799e-03, -1.22427166e-01, -4.79891896e-04, -1.75064266e-01,
        1.11599177e-01,  5.15314043e-02, -1.76547706e-01,  1.73693076e-01,
        1.20601654e-02, -9.45737958e-02,  2.99837291e-02, -2.94954255e-02,
       -1.16427243e-02, -2.93644071e-02,  2.36046970e-01,  3.26335430e-04,
       -6.34741783e-02, -1.41857266e-02,  3.37720215e-02,  4.02529240e-02,
       -2.67912745e-02, -7.81332627e-02,  6.36820868e-02,  2.41965115e-01,
        9.57147479e-02,  1.47209689e-02, -1.44766018e-01,  1.07569695e-02,
       -6.36568367e-02,  1.77488267e-01,  2.79973149e-01,  2.26736605e-01,
       -2.69324630e-02,  1.59850359e-01,  2.34070942e-02, -8.95017385e-03,
       -9.60781723e-02, -8.70623589e-02,  1.16312623e-01, -2.31613845e-01,
       -9.06762183e-02,  9.19840485e-02, -6.68512881e-02,  2.57165730e-02,
       -2.61168569e-01,  1.17108688e-01,  1.57850385e-02, -1.30072236e-03,
        2.16973305e-01,  1.21969432e-02, -1.33142516e-01,  9.84175205e-02,
       -1.01030469e-02,  

In [ ]:
res2 = word2vec.wv['boy'] -  word2vec.wv['girl'] + word2vec['woman']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
word2vec.wv.similar_by_vector(res2)

[('boy', 0.9745819568634033),
 ('hiding', 0.9569803476333618),
 ('woman', 0.954512894153595),
 ('dumped', 0.952126145362854),
 ('penned', 0.9520639777183533),
 ('named', 0.9479455947875977),
 ('lady', 0.9450726509094238),
 ('victim', 0.9439608454704285),
 ('mother', 0.9435360431671143),
 ('old', 0.9430907964706421)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [ ]:
# YOUR CODE HERE

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)` , actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it chooses many splits in the different sentences, choose some words as inputs $X$  and a word as output $y$ which it tries to predict, in the embedding space.

And as any neural network, Word2Vec has some hyperparameters. Let's check some. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [ ]:

word2vec_2 = Word2Vec(sentences=X_train, size = 90)

❓ **Question** ❓ Use the `word2vec.wv.key_to_index` attribute to display the size of the learnt vocabulary. On the other hand, compare it to the number of different words in `X_train`.

In [ ]:
import numpy as np
np.unique(X_train).shape

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


(2498,)

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been train on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec :  `min_count`. 

`min_count` is a integer that tells you how many occurences a given word should have to be learn in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vec that you have trained (you can choose any `vector_size` you want).

In [ ]:
word2vec_3 = Word2Vec(sentences = X_train, size =100,min_count = 7)

Remember that we say that word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Learn a new `word2vec_5` model with a `window` different than previously (default is 5).

In [ ]:
word2vec_5 = Word2Vec(sentences = X_train, size = 100, min_count =7,window= 7)

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the one that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here :

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [ ]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    embedded =[]
    try:
      for _ in sentence:
        embedded.append(word2vec.wv.get_vector(_))
    except Exception as e:
      pass

    return np.array(embedded)

    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

In [ ]:
embed_sentence(word2vec, example_missing_words).shape

(3, 100)

❓ **Question** ❓ Write a function that, given a list of sentence (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [ ]:
def embedding(word2vec, sentences):
    pass  # YOUR CODE HERE
    
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad them in order to have tensors that can be divided in batch sizes during the optimization. Store the padedd values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [ ]:
### YOUR CODE HERE

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)